In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 13 18:40:09 2022

@author: PC
"""

import json
import math
import os

import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm

def resize_image(image_path, desired_size=224):
    im = Image.open(image_path)
    #im = im.resize((desired_size, desired_size ), resample=Image.LANCZOS)
    
    return im

def load_image_ben_orig(path,resize=True,crop=False,norm255=True,keras=False):
    image = cv2.imread(path)
    
    
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
    image=cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) ,  10) ,-4 ,128)
    
    if norm255:
        return image/255
    elif keras:
        
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

def transform_image_ben(img,resize=True,crop=False,norm255=True,keras=False):  
    image=cv2.addWeighted( img,4, cv2.GaussianBlur( img , (0,0) ,  10) ,-4 ,128)
    
    if norm255:
        return image/255
    elif keras:
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
print(train_df.shape)


In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.uint8)


for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = Image.open(f'../input/cropped-degrees-dataset/cropped 0.1/{image_id}.jpg')


In [ ]:
y_train = pd.get_dummies(train_df['diagnosis']).values

print(x_train.shape)


print(y_train.shape)



In [ ]:
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))
print(y_train)
print(y_train_multi)




In [ ]:
x_trin, x_val, y_trin, y_val = train_test_split(
    x_train, y_train_multi, 
    test_size=0.2,
    random_state=2019
)


In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  
        rotation_range=360,
        featurewise_std_normalization = True,
        fill_mode='constant',
        cval=0.,  
        horizontal_flip=True,  
        vertical_flip=True, 
    )

data_generator = create_datagen().flow(x_trin, y_trin, batch_size=BATCH_SIZE, seed=2019)




In [ ]:
densenet = DenseNet121(
    weights="../input/densenet-keras/DenseNet-BC-121-32-no-top.h5" ,
    include_top=False,
    input_shape=(224,224,3)
)

GAP_layer = layers.GlobalAveragePooling2D()
drop_layer = layers.Dropout(0.5)
dense_layer = layers.Dense(5, activation='sigmoid', name='final_output')





In [ ]:
from tensorflow.keras.models import Model

def build_model_functional():
    base_model = densenet
    #base_model.trainable=False
    x = GAP_layer(base_model.layers[-1].output)
    x = drop_layer(x)
    final_output = dense_layer(x)
    model = Model(base_model.layers[0].input, final_output)
    
    return model




model = build_model_functional()
#model.summary()




In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6), 
        loss='binary_crossentropy', 
        metrics=['accuracy','AUC']
    )

In [ ]:
bucket_num = 3
div = round(train_df.shape[0]/bucket_num)

In [ ]:
df_init = {
    'val_loss': [0.0],
    'val_acc': [0.0],
    'loss': [0.0], 
    'acc': [0.0],
    'bucket': [0.0]
}
results = pd.DataFrame(df_init)

In [ ]:
import math
import gc

epochs = [5,5,5]
checkpoint_filepath = 'weights.{bucket_num:02d}-{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [ ]:
for i in range(0,bucket_num):
    if i != (bucket_num-1):
        print("Bucket Nr: {}".format(i))
        
        history = model.fit_generator(
                        data_generator,
                        steps_per_epoch=x_trin.shape[0] / BATCH_SIZE,
                        epochs=epochs[i],
                        validation_data=(x_val, y_val),
                        callbacks=[model_checkpoint_callback],
                        )
        
        dic = history.history
        df_model = pd.DataFrame(dic)
        df_model['bucket'] = i
    else:
        print("Bucket Nr: {}".format(i))
        
        history = model.fit_generator(
                        data_generator,
                        steps_per_epoch=x_trin.shape[0] / BATCH_SIZE,
                        epochs=epochs[i],
                        validation_data=(x_val, y_val),
                        callbacks=[model_checkpoint_callback],
                        )
        
        dic = history.history
        df_model = pd.DataFrame(dic)
        df_model['bucket'] = i

    results = results.append(df_model)
    
    gc.collect()
    
    print('-'*40)

In [ ]:
print(results)

In [ ]:
df_reset=results.reset_index()
print(df_reset)

In [ ]:
plt.plot(df_reset.val_accuracy)

In [ ]:
plt.plot(df_reset.val_auc)

In [ ]:
model.save("model_0.1.h5")

In [ ]:
import sklearn.metrics as met

y_val1=y_val>0.5
y_val2=y_val1*1
y_pred=model.predict(x_val)
y_pred1=y_pred>0.5
y_pred2=y_pred1*1
acc_score=met.accuracy_score(np.argmax(y_val, axis=1),np.argmax(y_pred, axis=1))
acc_score1=met.accuracy_score(y_val,y_pred2)
print(acc_score)
print(acc_score1)

In [ ]:
auc=met.roc_auc_score(np.argmax(y_val, axis=1),np.argmax(y_pred, axis=1))
auc1=met.roc_auc_score(y_val,y_pred2)
#print(auc)
print(auc1)